# 基于Datalog的符号推理  
---
- 本体推理的局限：  
    1. 主要实现了基于本体推理概念描述的推理，无法支持规则型知识的推理
    2. 用户无法定义自己的推理过程  
- 引入规则推理
    1. 可以根据特定的场景定制规则，以实现用户自定义的推理过程  
    2. Datalog语言可以结合本体推理和规则推理  
    3. Datalog是面向知识库和数据库设计的逻辑语言，表达能力与OWL相当，支持递归，同时便于撰写规则，实现规则推理
    <img src="../img/10.png">  
    
---
## Datalog基本语法  
<h4>原子(Atom)</h4>  

- p(t_1, t_2, …, t_n)  
- 其中p是谓词，n是目数，t_i是项（变量或常量）
- 例如：has_child(X, Y)  


<h4>规则(Rule)</h4>  

- H:-B_1, B_2, …, B_m  
- 由原子构建，其中H是头部原子，B_1, B_2, …, B_m是体部原子  
- 例如：has_child(X, Y): -has_son(X, Y).

<h4>事实(Fact)</h4>  

- F(c_1, c_2, c_n):-
- 没有体部且没有变量的规则
- 例如：has_child(Alice, Bob):-

<h4>Datalog程序是规则的集合</h4>  

- 例如：Datalog程序P:  
       has_child(X, Y):-has_son(X, Y)  
       has_child(Alice, Bob):-  

# 基于产生式规则的推理  
---
- 产生式系统  
    - 一种前向推理系统，可以按照一定机制执行规则从而达到某些目标，与一阶逻辑类似，也有区别  
    - 应用  
        - 自动规划  
        - 专家系统  
- 产生系统的组成  
    - 事实集合（Working Memory）  
    - 产生式/规则集合  
    - 推理引擎  
---
- 事实集/运行内存（Working Memory，WM）  
    - 事实（WME）的集合  
    - 用于存储当前系统中的所有事实  
- 事实（Working Memory Element， WME）  
    - 描述对象  
        - 形如（type attr_1:val_1 attr_2:val_2…attr_n:val_n），其中type，attr_i，val_i均为原子（常量）  
        - e.g:（student name:Alice age:24）  
    - 描述关系（Refication）  
        - e.g:olderThan John Alice  
---
- 产生式集合（Production Memory，PM）  
    - 产生式规则的集合  
- 产生式  
    - IF conditions Then actions  
    - conditions 是由条件组成的集合，又称LHS(Left Hand Side)  
    - actions 是由动作组成的序列， 又称RHS(Right Hand Side)  
    - e.g:IF（Student name：x）  
          Then ADD(Person name:x)  
          **or**  
          (Student name:x) --> ADD(Person name:x)  
          如果有一个学生名为?x, 那么向事实集中加入一个事实，表示有一个名为?x的人  
---
- LHS  
    - 条件（condition）的集合，各条件之间是**且**的关系  
    - 当LHS中所有条件**均**被满足，则该规则触发  
    - 每个条件形如（type attr_1: spec_1 attr_2:spec_2… attr_n:spec_n）  
        - 其中spec_i 表示对attr_i的约束，形式可取下列中的一种  
            - 原子，如：Alice       (person name: Alice)  
            - 变量，如：x（斜体）    (person name: x）  
            - 表达式， 如：[n + 4]  (person age: [n + 4])  
            - 布尔测试，如：{> 10}  (person age:{> 10})  
            - 约束的**与、或、非**操作  
- RHS  
    - 动作（action）的序列，执行时依次执行  
    - 动作的种类如下：  
        - ADD pattern  
            - 向WM中加入形如pattern的WME  
        - REMOVE i  
            - 从WM中移除当前规则第i个条件匹配的WME  
        - MODIFY i（attr spec）  
            - 对于当前规则第i个条件匹配的WME，将其对应于attr属性的值改为spec  
---   
<font color = red>产生式系统 = 事实集 + 产生式集合 +推理引擎</font>  
- 推理引擎  
    - 控制系统的执行  
        - **模式匹配：用规则的条件部分匹配事实集中的事实，整个LHS都被满足的规则被触发，并被加入议程（agenda）**  
        - **解决冲突：按一定的策略从被触发的规则中选择一条**  
        - 执行动作：执行被选择出来的规则的RHS，从而对WM进行一定的操作
<img src="../img/11.png">  
- **模式匹配（核心）**  
    - 用每条规则的条件部分匹配当前WM  
---

## RETE算法
---
<img src="../img/12.png">  

- 将产生式的LHS组织成判别网络形式  

- 核心思想是用分离的匹配项构造匹配网络，同时缓存中间结果。以空间换时间  

<img src="../img/13.png">  

---
- Rete 算法是一种启发式算法,不同规则之间往往含有相同的模式,因此在Rete网络中可以共享不同规则的条件部分。如果Rete网络中的某一个条件node被N条规则共享,则算法在此节点上效率会提高N倍  
- Rete 算法由于采用AlphaMemory和BetaMemory来存储事实,当事实集合变化不大时,保存在alpha和beta节点中的状态不需要太多变化,避免了大量的重复计算,提高了匹配效率  
- 从Rete网络可以看出,Rete匹配速度与规则数目无直接关系,这是因为事实只有满足本节点才会继续向下沿网络传递

<h4>常见的基于符号的知识图谱推理方法有基于OWL的本体推理,基于Datalog的规则推理和比较传统的基于Rete算法的产生式规则推理。  </h4>

<h4>基于符号表示的推理最大的优势是精确并具有可解释性,因而对那些规模可控,对知识表示的精确度要求比较高的场景更加有用。</h4>

<h4>这类基于符号表示的推理都是基于演绎逻辑的推理,对知识的表示和描述要求比较高，这增加了知识获取的难度,同时在知识库规模比较大时,推理的鲁棒性和效率都会降低。因此,当前更多的研究是基于大数据的推理实现.</h4>